In [167]:
import numpy as np
import pandas as pd
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import fuzzymatcher

In [168]:
data_dir = Path('D:\Project\Project_THDL\Book data')

In [169]:
df_recomadation_book = pd.read_csv(data_dir / 'Book recomadation dataset/data.csv')


In [226]:
df_genPre_book = pd.read_csv(data_dir / 'Book genre prediction/Books.csv')

#format column name
df_genPre_book.drop(['ISBN'],axis=1, inplace=True)
df_genPre_book.rename(columns={'Book-Title':'title'}, inplace=True)
df_genPre_book.rename(columns={'Book-Author':'author'}, inplace=True)
df_genPre_book.rename(columns={'Year-Of-Publication':'year'}, inplace=True)
df_genPre_book.rename(columns={'Publisher':'publisher'}, inplace=True)

In [171]:
df_goodRead_book = pd.read_csv(data_dir / 'Book Goodreads/books.csv')


In [172]:
df_Crossing_book = pd.read_csv(data_dir / 'Book Crossing/Books Data with Category Language and Summary/Preprocessed_data.csv')



In [173]:
df_Crossing_book.drop(df_Crossing_book[df_Crossing_book['isbn']=='0751352497'].index, axis=0, inplace=True)
df_Crossing_book.drop(['img_s',
                       'img_m',
                       'img_l', 
                       'location',
                       'isbn'],axis=1, inplace=True)
df_Crossing_book.rename(columns={'book_author':'author'}, inplace=True)
df_Crossing_book.rename(columns={'book_title':'title'}, inplace=True)

In [174]:
df_depository_book = pd.read_csv(data_dir / 'Book depository/dataset.csv')

In [175]:
df_depository_book.drop([467665], axis=0, inplace=True)
df_depository_book.drop(['dimension-x', 
                         'dimension-y', 
                         'dimension-z', 
                         'edition',
                         'edition-statement',
                         'for-ages',
                         'illustrations-note',
                         'image-checksum',
                         'image-path',
                         'image-url',
                         'imprint',
                         'index-date',
                         'publication-place',
                         'url',
                         'weight',
                         'isbn10',
                         'isbn13'
                         ], axis=1, inplace=True)
df_depository_book.rename(columns={'authors':'author'}, inplace=True)

In [176]:
#df_goodRead_book.info() #abandon
#df_genPre_book.info() 
#df_recomadation_book.info()
df_depository_book.info()
df_Crossing_book.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 1109382 entries, 0 to 1109382
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   author            1109382 non-null  object 
 1   bestsellers-rank  642540 non-null   float64
 2   categories        1109382 non-null  object 
 3   description       1029295 non-null  object 
 4   format            1102760 non-null  float64
 5   id                1109382 non-null  int64  
 6   lang              1048975 non-null  object 
 7   publication-date  1106779 non-null  object 
 8   rating-avg        669252 non-null   float64
 9   rating-count      669252 non-null   float64
 10  title             1109382 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 101.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 1031174 entries, 0 to 1031174
Data columns (total 14 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               -----

In [177]:
#####################################################
#process book recomadation and book genre prediction#
#####################################################

In [227]:
df_genPre_book['recomadation_book_exists'] = np.where(df_genPre_book['title'].isin(list(df_recomadation_book['title'].unique())),1,0)

df_genPre_book[df_genPre_book['recomadation_book_exists']==1]

,title,author,year,publisher,recomadation_book_exists
13,Jane Doe,R. J. Kaiser,1999,Mira Books,1
18,The Testament,John Grisham,1999,Dell,1
38,Seabiscuit: An American Legend,LAURA HILLENBRAND,2002,Ballantine Books,1
41,Pride and Prejudice,Jane Austen,1983,Bantam,1
46,From the Corner of His Eye,Dean Koontz,2001,Bantam Books,1
...,...,...,...,...,...
270659,Lightning,Patricia Potter,1992,Bantam Books,1
270739,Hush Little Baby,Sylvia Long,2002,Chronicle Books,1
270926,Radiance,Anne Maybury,1980,Ace Books,1
271055,Anthem,Ayn Rand,1966,Caxton Press,1


In [179]:
df_recomadation_book['genPre_book_exists'] = np.where(df_recomadation_book['title'].isin(list(df_genPre_book['title'].unique())),1,0)

df_recomadation_book[df_recomadation_book['genPre_book_exists']==1]

,index,title,genre,summary,genPre_book_exists
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben following the event...,1
5,5,Thendara House,fantasy,The novel concerns the dwelling of the Darkovan Order of the Renu...,1
6,6,The Thief,fantasy,"Gen is released from prison by the magus, the king's scholar. The...",1
11,11,Queen of Demons,fantasy,"In the introduction, the current King of the Isles, Valence III, ...",1
15,15,Always Coming Home,fantasy,The book weaves around the story of a Kesh woman called Stone Tel...,1
...,...,...,...,...,...
4638,4638,Perdido Street Station,fantasy,"Beneath the towering bleached ribs of a dead, ancient beast lies t...",1
4648,4648,The Time Traveler's Wife,fantasy,"This is the extraordinary love story of Clare and Henry, who met w...",1
4649,4649,Beauty: A Retelling of the Story of Beauty and the Beast,fantasy,A strange imprisonment...\n\nBeauty has never liked her nickname. ...,1
4653,4653,Charlie and the Chocolate Factory,fantasy,Charlie Bucket's wonderful adventure begins when he finds one of M...,1


In [180]:
df_genPre_no_recomadation = pd.DataFrame(df_genPre_book[df_genPre_book['recomadation_book_exists']==0]['title'].unique()).set_axis(['title'],axis=1)
df_genPre_no_recomadation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240474 entries, 0 to 240473
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   240474 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


In [181]:
df_recomadation_no_genPre = pd.DataFrame(df_recomadation_book[df_recomadation_book['genPre_book_exists']==0]['title'].unique()).set_axis(['title'],axis=1)
df_recomadation_no_genPre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2639 non-null   object
dtypes: object(1)
memory usage: 20.7+ KB


In [182]:
# Fuzzy matching
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [183]:
#fuzzy_match_book_name = fuzzy_merge(df_recomadation_no_genPre, df_genPre_no_recomadation, 'title', 'title', threshold=80, limit=1)
#fuzzy_match_book_name.to_csv(data_dir / 'fuzzy match recom vs genPre.csv', index=False)

In [184]:
#fuzzymatcher_match_recom_vs_genPre = fuzzymatcher.fuzzy_left_join(df_recomadation_no_genPre, df_genPre_no_recomadation, left_on='title', right_on='title')
#fuzzymatcher_match_recom_vs_genPre.to_csv(data_dir / 'fuzzy match recom vs genPre (fuzzymatcher).csv', index=False)

In [185]:
fuzzymatcher_match_recom_vs_genPre = pd.read_csv(data_dir / 'fuzzy match recom vs genPre (fuzzymatcher).csv')

In [186]:
fuzzymatcher_match_recom_vs_genPre.sort_values(by=['best_match_score'], ascending=False).head(10)

,best_match_score,__id_left,__id_right,title_left,title_right
1701,4.036427,1701_left,159160_right,How the Irish Saved Civilization: The Untold Story of Ireland's He...,How the Irish Saved Civilization : The Untold Story of Ireland's H...
1825,3.597682,1825_left,143600_right,The Gift of Therapy: An Open Letter to a New Generation of Therapi...,The Gift of Therapy : An Open Letter to a New Generation of Therap...
2343,3.055182,2343_left,49844_right,I'm a Stranger Here Myself: Notes on Returning to America After Tw...,I'm a Stranger Here Myself : Notes on Returning to America After T...
1677,2.907667,1677_left,31334_right,"The Devil in the White City: Murder, Magic, and Madness at the Fai...","The Devil in the White City: Murder, Magic, and Madness at the Fai..."
1688,2.737315,1688_left,2066_right,Lies My Teacher Told Me: Everything Your American History Textbook...,Lies My Teacher Told Me : Everything Your American History Textboo...
1690,2.711565,1690_left,3322_right,"The Professor and the Madman: A Tale of Murder, Insanity and the M...","The Professor and the Madman: A Tale of Murder, Insanity, and the ..."
1945,2.568821,1945_left,13617_right,"""Surely You're Joking, Mr. Feynman!"": Adventures of a Curious Char...","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Cha..."
1679,2.412883,1679_left,13428_right,The Rise and Fall of the Third Reich: A History of Nazi Germany,The Rise and Fall of the Third Reich : A History of Nazi Germany
2347,2.339485,2347_left,12875_right,Into Thin Air: A Personal Account of the Mount Everest Disaster,Into Thin Air : A Personal Account of the Mount Everest Disaster
1996,2.301154,1996_left,57458_right,Billions & Billions: Thoughts on Life and Death at the Brink of th...,Billions and Billions: Thoughts on Life and Death at the Brink of ...


In [187]:
fuzzymatcher_match_recom_vs_genPre.query('best_match_score <= 0.3').sort_values(by=['best_match_score'], ascending=False).head(10)

,best_match_score,__id_left,__id_right,title_left,title_right
499,0.299569,499_left,46492_right,Learning the World,The Crazy World of Learning to Drive
2334,0.299346,2334_left,55061_right,Vicious,Vicious Circle
35,0.299156,35_left,30183_right,Alice in Verse: The Lost Rhymes of Wonderland,Alice in Wonderland
70,0.299104,70_left,184544_right,Gate of Ivrel,Gate of Ivrel (DAW Books = sf)
67,0.298845,67_left,52624_right,The Lake of Tears,"The Lake of Tears (Deltora Quest, 2)"
964,0.298316,964_left,82905_right,Adam of the Road,Adam of the Road (Puffin Newberry Library)
2336,0.297986,2336_left,127632_right,The Collective,Murder in the Collective
121,0.297905,121_left,26081_right,Fablehaven: Rise of the Evening Star,The Evening Star
2184,0.297774,2184_left,191314_right,The Lost Man,Man Who Lost the War
204,0.297602,204_left,131797_right,The Deluge Drivers,"Deluge Drivers (Icerigger Trilogy, Book 3)"


In [188]:
fuzzymatcher_match_recom_vs_genPre.drop(fuzzymatcher_match_recom_vs_genPre[fuzzymatcher_match_recom_vs_genPre['best_match_score']<0.6].index, axis=0, inplace=True)

In [228]:
fuzzy_match_dict = dict(zip(fuzzymatcher_match_recom_vs_genPre['title_right'], fuzzymatcher_match_recom_vs_genPre['title_left']))

In [229]:
df_genPre_book['title'] = df_genPre_book['title'].replace(fuzzy_match_dict)

In [230]:
df_genPre_book['recomadation_book_exists'] = np.where(df_genPre_book['title'].isin(list(df_recomadation_book['title'].unique())),1,0)


In [192]:
df_genPre_book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   title                     271360 non-null  object
 1   author                    271358 non-null  object
 2   year                      271360 non-null  int64 
 3   publisher                 271358 non-null  object
 4   recomadation_book_exists  271360 non-null  int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 9.3+ MB


In [193]:
df_recomadation_book['genPre_book_exists'] = np.where(df_recomadation_book['title'].isin(list(df_genPre_book['title'].unique())),1,0)


In [194]:
df_recomadation_book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4657 entries, 0 to 4656
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               4657 non-null   int64 
 1   title               4657 non-null   object
 2   genre               4657 non-null   object
 3   summary             4657 non-null   object
 4   genPre_book_exists  4657 non-null   int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 163.9+ KB


In [195]:
df_recomadation_book.drop(df_recomadation_book[df_recomadation_book['genPre_book_exists']==0].index, axis=0, inplace=True)


In [196]:
###########################################
#process book depository and book crossing#
###########################################

In [197]:
df_Crossing_book['depository_book_exists'] = np.where(df_Crossing_book['title'].isin(list(df_depository_book['title'].unique())),1,0)

df_Crossing_book[df_Crossing_book['depository_book_exists']==1].info()


<class 'pandas.core.frame.DataFrame'>
Index: 273873 entries, 0 to 1031165
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              273873 non-null  int64  
 1   user_id                 273873 non-null  int64  
 2   age                     273873 non-null  float64
 3   rating                  273873 non-null  int64  
 4   title                   273873 non-null  object 
 5   author                  273873 non-null  object 
 6   year_of_publication     273873 non-null  float64
 7   publisher               273873 non-null  object 
 8   Summary                 273873 non-null  object 
 9   Language                273873 non-null  object 
 10  Category                273873 non-null  object 
 11  city                    270595 non-null  object 
 12  state                   268364 non-null  object 
 13  country                 264134 non-null  object 
 14  depository_book_exists  

In [235]:
df_Crossing_book.head(10)

,Unnamed: 0,user_id,age,rating,title,author,year_of_publication,publisher,Summary,Language,Category,city,state,country,depository_book_exists
0,0,2,18.0000,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,Provides an introduction to classical myths placing the addressed\...,en,['Social Science'],stockton,california,usa,1
15,15,8,34.7439,0,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],timmins,ontario,canada,1
16,16,2954,71.0000,8,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],wichita,kansas,usa,1
17,17,152827,40.0000,7,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],ottawa,ontario,canada,1
18,18,8,34.7439,0,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],timmins,ontario,canada,1
19,19,35704,53.0000,6,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],kansas city,missouri,usa,1
20,20,83160,65.0000,0,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],oregon city,oregon,usa,1
21,21,110912,36.0000,10,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],milpitas,california,usa,1
22,22,137688,46.0000,0,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],medford,new york,usa,1
23,23,157969,30.0000,8,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],denver,colorado,usa,1


In [198]:
df_depository_book['crossing_book_exists'] = np.where(df_depository_book['title'].isin(list(df_Crossing_book['title'].unique())),1,0)

df_depository_book[df_depository_book['crossing_book_exists']==1]

,author,bestsellers-rank,categories,description,format,id,lang,publication-date,rating-avg,rating-count,title,crossing_book_exists
30,[42],252306.0,"[2978, 2980]",TV Go Home began life in the late 1990s as an outrageously funny w...,1.0,9780571272198,en,2010-11-04 00:00:00,4.23,315.0,TV Go Home,1
141,"[192, 193]",357453.0,"[814, 3359, 833, 3378]","Our traditional Great Teacher teaches by telling, inspiring studen...",1.0,9780867094695,en,2000-03-03 00:00:00,3.86,117.0,Teaching with Your Mouth Shut,1
232,[309],247880.0,"[1520, 1530, 1710]",Humans have always sought to change their environment--building ho...,1.0,9780312141042,en,2000-09-05 00:00:00,3.55,344.0,The Existential Pleasures of Engineering,1
269,[376],2424475.0,"[2770, 2865]","The popular international telecast, It Is Written, heartily recomm...",1.0,9781572589537,en,2011-05-25 00:00:00,NaN,NaN,375 Meatless Recipes,1
484,[691],91235.0,"[1520, 1533, 1535, 3012]",Astronomy has never been a more popular pastime than it is today. ...,1.0,9780671664046,en,1988-06-15 00:00:00,3.62,16.0,How To Use An Astronomical Telescope,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1108948,[653819],3112983.0,"[3092, 2521, 2543, 2564]",The man who conceived of the Appalachian Trail believed that time ...,1.0,9781482446739,en,2016-08-15 00:00:00,NaN,NaN,The Appalachian Trail,1
1109075,[624266],NaN,"[3013, 3055]","Offers a compelling account of the dramatic story of Ruffian, hail...",1.0,9780345386021,en,1994-04-01 00:00:00,4.36,2571.0,Ruffian: Burning from the Start,1
1109118,[232677],NaN,[3055],NaN,1.0,9780026887588,NaN,1990-05-01 00:00:00,4.20,24654.0,King of the Wind,1
1109341,[35294],NaN,[3096],This planner has all you need to organize your life in 2019! Featu...,1.0,9781723791123,en,2018-09-18 00:00:00,NaN,NaN,I,1


In [199]:
df_Crossing_book_no_depos = pd.DataFrame(df_Crossing_book[df_Crossing_book['depository_book_exists']==0]['title'].unique()).set_axis(['title'],axis=1)
df_Crossing_book_no_depos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222199 entries, 0 to 222198
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   222199 non-null  object
dtypes: object(1)
memory usage: 1.7+ MB


In [200]:
df_depository_book_no_cross = pd.DataFrame(df_depository_book[df_depository_book['crossing_book_exists']==0]['title'].unique()).set_axis(['title'],axis=1)
df_depository_book_no_cross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 938669 entries, 0 to 938668
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   938669 non-null  object
dtypes: object(1)
memory usage: 7.2+ MB


In [201]:
df_depository_book_no_cross.drop(df_depository_book_no_cross.tail(500000).index, inplace=True)

In [202]:
df_depository_book_no_cross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438669 entries, 0 to 438668
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   438669 non-null  object
dtypes: object(1)
memory usage: 3.3+ MB


In [203]:
#fuzzymatcher_match_cross_vs_depos = fuzzymatcher.fuzzy_left_join(df_Crossing_book_no_depos, df_depository_book_no_cross, left_on='title', right_on='title')
#fuzzymatcher_match_cross_vs_depos.to_csv(data_dir / 'fuzzy match cross vs depos (fuzzymatcher).csv', index=False)

In [204]:
fuzzymatcher_match_cross_vs_depos = pd.read_csv(data_dir / 'fuzzy match cross vs depos (fuzzymatcher).csv')

In [205]:
fuzzymatcher_match_cross_vs_depos.query('best_match_score <= 0.55').sort_values(by=['best_match_score'], ascending=False).head(10)

,best_match_score,__id_left,__id_right,title_left,title_right
196088,0.549975,196088_left,242738_right,Beginner's Guide to Practical Astrology,The Beginner's Guide to Astrology : A Practical Introduction to th...
90962,0.549970,90962_left,72501_right,Bob Dylan: Behind the Shades Revisited,Bob Dylan Revisited
40820,0.549939,40820_left,328175_right,Ray Bradbury Presents Dinosaur World,Ray Bradbury Dinosaur Tales
105768,0.549925,105768_left,380839_right,Spark of Life,Spark Of Life
14441,0.549913,14441_left,84002_right,Holographic Universe,The Holographic Universe
217064,0.549910,217064_left,28699_right,At the Root of This Longing: Reconciling a Spiritual Hunger and a ...,At the Root of this Longing
64451,0.549890,64451_left,325166_right,Musical Moment : And Other Stories,The Musical Brain : And Other Stories
16932,0.549871,16932_left,210824_right,Machiavelli's The Prince (Cliffs Notes),"Machiavelli's ""The Prince"""
128173,0.549845,128173_left,363143_right,Mundo Del Fin Del Mundo (Andanzas),Mundo del fin del mundo
30258,0.549814,30258_left,263742_right,The Eye of Horus,Guardians : The Eye of Horus


In [206]:
fuzzymatcher_match_cross_vs_depos.drop(fuzzymatcher_match_cross_vs_depos[fuzzymatcher_match_cross_vs_depos['best_match_score']<0.55].index, axis=0, inplace=True)

In [207]:
fuzzy_match_dict_2 = dict(zip(fuzzymatcher_match_cross_vs_depos['title_left'], fuzzymatcher_match_cross_vs_depos['title_right']))

In [208]:
df_Crossing_book['title'] = df_Crossing_book['title'].replace(fuzzy_match_dict_2)

In [209]:
df_Crossing_book['depository_book_exists'] = np.where(df_Crossing_book['title'].isin(list(df_depository_book['title'].unique())),1,0)
df_Crossing_book[df_Crossing_book['depository_book_exists'] ==1 ]

,Unnamed: 0,user_id,age,rating,title,author,year_of_publication,publisher,Summary,Language,Category,city,state,country,depository_book_exists
0,0,2,18.0000,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,Provides an introduction to classical myths placing the addressed\...,en,['Social Science'],stockton,california,usa,1
15,15,8,34.7439,0,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],timmins,ontario,canada,1
16,16,2954,71.0000,8,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],wichita,kansas,usa,1
17,17,152827,40.0000,7,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],ottawa,ontario,canada,1
18,18,8,34.7439,0,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],timmins,ontario,canada,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031148,1031148,278648,34.7439,5,The Christmas Spirit,Patricia Wynn,1996.0,Ivy Books,"Taking human form as part of a wager, mischievous elf Trudy lures ...",en,['Fiction'],las vegas,nevada,usa,1
1031151,1031151,278713,63.0000,8,Orson Welles,Barbara Leaming,1985.0,Penguin USA,"Based on two years of interviews and research, this biography port...",en,['Biography & Autobiography'],albuquerque,new mexico,usa,1
1031164,1031164,278843,28.0000,0,Crafts and Craft Shows : How to Make Money,Phil Kadubec,2000.0,Watson-Guptill Publications,"Craftspeople, hobby-ists, and crafts-related businesses will find ...",en,['Crafts & Hobbies'],pismo beach,california,usa,1
1031165,1031165,278843,28.0000,0,The Queen's Gambit,Walter Tevis,1996.0,Texas Bookman,"Engaging and fast-paced, The Queen&#39;s Gambit speeds to a conclu...",en,['Fiction'],pismo beach,california,usa,1


In [210]:
df_depository_book['crossing_book_exists'] = np.where(df_depository_book['title'].isin(list(df_Crossing_book['title'].unique())),1,0)

df_depository_book[df_depository_book['crossing_book_exists']==1]

,author,bestsellers-rank,categories,description,format,id,lang,publication-date,rating-avg,rating-count,title,crossing_book_exists
30,[42],252306.0,"[2978, 2980]",TV Go Home began life in the late 1990s as an outrageously funny w...,1.0,9780571272198,en,2010-11-04 00:00:00,4.23,315.0,TV Go Home,1
41,[55],39840.0,"[750, 1532, 2770, 2812, 2813, 2820]",'Each part of your body will be working perfectly as a harmonious ...,1.0,9781561706020,en,2004-08-27 00:00:00,4.13,525.0,Love Your Body : A Positive Affirmation Guide for Loving and Appre...,1
54,[],2718403.0,[2980],This compilation includes the best of Angus Deaton's last two seri...,3.0,9780563496243,en,2003-10-06 00:00:00,4.00,3.0,"""Have I Got News for You""",1
56,[74],75625.0,"[2663, 635, 719, 1521, 1531, 1532, 1593, 1599, 2985]",Dr Jacob Bronowksi's The Ascent of Man traces the development of h...,1.0,9781849901154,en,2013-04-01 00:00:00,4.19,5749.0,The Ascent Of Man,1
112,"[147, 148]",117916.0,"[2649, 1531, 1566, 1719, 1852, 1862]",The authoritative masterpiece (L. A. Times) on the Apollo space pr...,1.0,9780143112358,en,2009-10-06 00:00:00,4.47,5898.0,A Man on the Moon : The Voyages of the Apollo Astronauts,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1108948,[653819],3112983.0,"[3092, 2521, 2543, 2564]",The man who conceived of the Appalachian Trail believed that time ...,1.0,9781482446739,en,2016-08-15 00:00:00,NaN,NaN,The Appalachian Trail,1
1109075,[624266],NaN,"[3013, 3055]","Offers a compelling account of the dramatic story of Ruffian, hail...",1.0,9780345386021,en,1994-04-01 00:00:00,4.36,2571.0,Ruffian: Burning from the Start,1
1109118,[232677],NaN,[3055],NaN,1.0,9780026887588,NaN,1990-05-01 00:00:00,4.20,24654.0,King of the Wind,1
1109341,[35294],NaN,[3096],This planner has all you need to organize your life in 2019! Featu...,1.0,9781723791123,en,2018-09-18 00:00:00,NaN,NaN,I,1


In [211]:
df_Crossing_book.drop(df_Crossing_book[df_Crossing_book['depository_book_exists']==0].index, axis=0, inplace=True)

In [212]:
#########################################
#process genre prediction and depository#
#########################################

In [213]:
df_depository_book['genPre_book_exists'] = np.where(df_depository_book['title'].isin(list(df_genPre_book['title'].unique())),1,0)

df_depository_book[df_depository_book['genPre_book_exists']==1]

,author,bestsellers-rank,categories,description,format,id,lang,publication-date,rating-avg,rating-count,title,crossing_book_exists,genPre_book_exists
30,[42],252306.0,"[2978, 2980]",TV Go Home began life in the late 1990s as an outrageously funny w...,1.0,9780571272198,en,2010-11-04 00:00:00,4.23,315.0,TV Go Home,1,1
141,"[192, 193]",357453.0,"[814, 3359, 833, 3378]","Our traditional Great Teacher teaches by telling, inspiring studen...",1.0,9780867094695,en,2000-03-03 00:00:00,3.86,117.0,Teaching with Your Mouth Shut,1,1
232,[309],247880.0,"[1520, 1530, 1710]",Humans have always sought to change their environment--building ho...,1.0,9780312141042,en,2000-09-05 00:00:00,3.55,344.0,The Existential Pleasures of Engineering,1,1
269,[376],2424475.0,"[2770, 2865]","The popular international telecast, It Is Written, heartily recomm...",1.0,9781572589537,en,2011-05-25 00:00:00,NaN,NaN,375 Meatless Recipes,1,1
484,[691],91235.0,"[1520, 1533, 1535, 3012]",Astronomy has never been a more popular pastime than it is today. ...,1.0,9780671664046,en,1988-06-15 00:00:00,3.62,16.0,How To Use An Astronomical Telescope,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108948,[653819],3112983.0,"[3092, 2521, 2543, 2564]",The man who conceived of the Appalachian Trail believed that time ...,1.0,9781482446739,en,2016-08-15 00:00:00,NaN,NaN,The Appalachian Trail,1,1
1109075,[624266],NaN,"[3013, 3055]","Offers a compelling account of the dramatic story of Ruffian, hail...",1.0,9780345386021,en,1994-04-01 00:00:00,4.36,2571.0,Ruffian: Burning from the Start,1,1
1109118,[232677],NaN,[3055],NaN,1.0,9780026887588,NaN,1990-05-01 00:00:00,4.20,24654.0,King of the Wind,1,1
1109341,[35294],NaN,[3096],This planner has all you need to organize your life in 2019! Featu...,1.0,9781723791123,en,2018-09-18 00:00:00,NaN,NaN,I,1,1


In [231]:
df_genPre_book['depository_book_exists'] = np.where(df_genPre_book['title'].isin(list(df_depository_book['title'].unique())),1,0)
df_genPre_book[df_genPre_book['depository_book_exists']==1]

,title,author,year,publisher,recomadation_book_exists,depository_book_exists
0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0,1
2,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,0,1
4,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,0,1
5,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,0,1
13,Jane Doe,R. J. Kaiser,1999,Mira Books,1,1
...,...,...,...,...,...,...
271294,The Jungle Books and Just So Stories,Kipling Rudyard,1986,Bantam Classics,0,1
271315,All the Queen's Men,Nick Elwood,1999,Gay Men's Press,0,1
271316,Singularity Sky,Charles Stross,2004,Ace,1,1
271328,Introductory Digital Signal Processing with Computer Applications,Paul A. Lynn,1989,John Wiley &amp; Sons,0,1


In [215]:
df_genPre_book_no_depos = pd.DataFrame(df_genPre_book[df_genPre_book['depository_book_exists']==0]['title'].unique()).set_axis(['title'],axis=1)

In [216]:
df_depository_book_no_genPre = pd.DataFrame(df_depository_book[df_depository_book['genPre_book_exists']==0]['title'].unique()).set_axis(['title'],axis=1)
df_depository_book_no_genPre.drop(df_depository_book_no_genPre.tail(500000).index, inplace=True)

In [217]:
df_genPre_book_no_depos.info()
df_depository_book_no_genPre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223195 entries, 0 to 223194
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   223195 non-null  object
dtypes: object(1)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438623 entries, 0 to 438622
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   438623 non-null  object
dtypes: object(1)
memory usage: 3.3+ MB


In [218]:
#fuzzymatcher_match_genPre_vs_depos = fuzzymatcher.fuzzy_left_join(df_genPre_book_no_depos, df_depository_book_no_genPre, left_on='title', right_on='title')
#fuzzymatcher_match_genPre_vs_depos.to_csv(data_dir / 'fuzzy match genPre vs depos (fuzzymatcher).csv', index=False)

In [219]:
fuzzymatcher_match_genPre_vs_depos = pd.read_csv(data_dir / 'fuzzy match genPre vs depos (fuzzymatcher).csv')
fuzzymatcher_match_cross_vs_depos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19446 entries, 1 to 222197
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   best_match_score  17236 non-null  float64
 1   __id_left         19446 non-null  object 
 2   __id_right        17236 non-null  object 
 3   title_left        19446 non-null  object 
 4   title_right       17236 non-null  object 
dtypes: float64(1), object(4)
memory usage: 911.5+ KB


In [220]:
fuzzymatcher_match_genPre_vs_depos.sort_values(by=['best_match_score'], ascending=True).head(10)

,best_match_score,__id_left,__id_right,title_left,title_right
43467,-5.503646,43467_left,118597_right,"What She Saw in Roger Mancuso, Gunter Hopstock, Jason Barry Gold, ...","A Study Guide for Anonymous's ""Tain Bo Cuailnge"""
106856,-5.340958,106856_left,310448_right,"Akbar Behkalam: Bewegung und VerÃ¤nderung, Bilder und Zeichnungen,...",Zot! : The Complete Black and White Collection: 1987-1991
180017,-5.178394,180017_left,1766_right,Die 13 1/2 Leben des KÃ¤pt'n BlaubÃ¤r: Die halben Lebenserinnerung...,Counting Your Way Through 1-2-3 : Books and Activities
138530,-4.963348,138530_left,3580_right,Blauer Montag.,"Tierarzt Praxisplaner Terminplaner : 2 Spalten pro Tag, 7-20 Uhr, ..."
216824,-4.695743,216824_left,106619_right,"Â°Iyunim bi-Megilat Teman: Kehilat TsanÂ°a, ShabtaÂ®ut, mishpahat ...",B.El.Ed. Entrance Exam Guide
107469,-4.484636,107469_left,201637_right,SHOW ME THE MAGIC : My Adventures in Life and Hollywood with Peter...,Life in the Key of Allegra : A Personal and Professional Journey w...
186178,-4.319507,186178_left,89655_right,Celinas Tochter.,Herzlichen Gluckwunsch zum 18. Maria : Das mit Vornamen personalis...
42539,-4.319507,42539_left,89655_right,Merkels Tochter.,Herzlichen Gluckwunsch zum 18. Maria : Das mit Vornamen personalis...
53622,-4.319507,53622_left,89655_right,Belsazars Tochter.,Herzlichen Gluckwunsch zum 18. Maria : Das mit Vornamen personalis...
75662,-4.093159,75662_left,358808_right,Marimba,30 Easy Play Songs every parent/grandparent can play for kids even...


In [221]:
fuzzymatcher_match_genPre_vs_depos.query('best_match_score <= 0.8').sort_values(by=['best_match_score'], ascending=False).head(10)

,best_match_score,__id_left,__id_right,title_left,title_right
189234,0.799995,189234_left,417616_right,Along the Garden Path: More Quilters and Their Gardens,Through the Garden Gate : Quilters and Their Gardens
108255,0.799854,108255_left,210973_right,With Every Breath : With Every Breath,With Every Breath
130102,0.799669,130102_left,61432_right,Global Flyer: Around the World in 80 Days,Around the World In 80 Days
114428,0.799633,114428_left,97932_right,"Vacation Under The Volcano (Magic Tree House 13, paper)",Study Guide Student Workbook for The Magic Tree House Vacation Und...
63032,0.799631,63032_left,20848_right,The Shrinking of Treehorn,The Shrinking of Treehorn (50th Anniversary Edition)
166390,0.799555,166390_left,366043_right,Leavenworth Case,The Leavenworth Case
191771,0.799544,191771_left,326586_right,Misterio En El Castillo Del Terror/the Secret of Terror Castle,Los tres investigadores. Misterio en el castillo del terror
123822,0.799484,123822_left,384301_right,A Prayer for Owen Meany (Modern Library),A Prayer For Owen Meany
103686,0.799466,103686_left,322217_right,March to the Stars,March To The Stars
181948,0.799448,181948_left,69457_right,Lord and His Prayer,The Lord and His Prayer


In [222]:
fuzzymatcher_match_genPre_vs_depos.drop(fuzzymatcher_match_genPre_vs_depos[fuzzymatcher_match_genPre_vs_depos['best_match_score']<0.7].index, axis=0, inplace=True)
fuzzymatcher_match_genPre_vs_depos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14883 entries, 1 to 223185
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   best_match_score  12661 non-null  float64
 1   __id_left         14883 non-null  object 
 2   __id_right        12661 non-null  object 
 3   title_left        14883 non-null  object 
 4   title_right       12661 non-null  object 
dtypes: float64(1), object(4)
memory usage: 697.6+ KB


In [223]:
fuzzy_match_dict_3 = dict(zip(fuzzymatcher_match_genPre_vs_depos['title_left'], fuzzymatcher_match_genPre_vs_depos['title_right']))

In [232]:
df_genPre_book['title'] = df_genPre_book['title'].replace(fuzzy_match_dict_3)

In [233]:
df_genPre_book['depository_book_exists'] = np.where(df_genPre_book['title'].isin(list(df_depository_book['title'].unique())),1,0)
df_genPre_book[df_genPre_book['depository_book_exists']==1]

,title,author,year,publisher,recomadation_book_exists,depository_book_exists
0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0,1
2,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,0,1
3,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999,Farrar Straus Giroux,0,1
4,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,0,1
5,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,0,1
...,...,...,...,...,...,...
271316,Singularity Sky,Charles Stross,2004,Ace,1,1
271328,Introductory Digital Signal Processing with Computer Applications,Paul A. Lynn,1989,John Wiley &amp; Sons,0,1
271336,Hausaufgabenbetreuung an Ganztagsschulen. Zwischen Anspruch und Wi...,GÃ¼nther Unser,1995,Bundeszentrale fÃ¼r Politische Bildung,0,1
271348,Slow Food : The Case for Taste,Carlo Petrini,2003,Columbia University Press,0,1


In [234]:
df_depository_book['genPre_book_exists'] = np.where(df_depository_book['title'].isin(list(df_genPre_book['title'].unique())),1,0)
df_depository_book[df_depository_book['genPre_book_exists']==1]

,author,bestsellers-rank,categories,description,format,id,lang,publication-date,rating-avg,rating-count,title,crossing_book_exists,genPre_book_exists
30,[42],252306.0,"[2978, 2980]",TV Go Home began life in the late 1990s as an outrageously funny w...,1.0,9780571272198,en,2010-11-04 00:00:00,4.23,315.0,TV Go Home,1,1
41,[55],39840.0,"[750, 1532, 2770, 2812, 2813, 2820]",'Each part of your body will be working perfectly as a harmonious ...,1.0,9781561706020,en,2004-08-27 00:00:00,4.13,525.0,Love Your Body : A Positive Affirmation Guide for Loving and Appre...,1,1
56,[74],75625.0,"[2663, 635, 719, 1521, 1531, 1532, 1593, 1599, 2985]",Dr Jacob Bronowksi's The Ascent of Man traces the development of h...,1.0,9781849901154,en,2013-04-01 00:00:00,4.19,5749.0,The Ascent Of Man,1,1
112,"[147, 148]",117916.0,"[2649, 1531, 1566, 1719, 1852, 1862]",The authoritative masterpiece (L. A. Times) on the Apollo space pr...,1.0,9780143112358,en,2009-10-06 00:00:00,4.47,5898.0,A Man on the Moon : The Voyages of the Apollo Astronauts,1,1
132,[176],68305.0,"[214, 228, 684, 1531, 1600, 1601]","In 1962, Maurice Wilkins, Francis Crick, and James Watson received...",1.0,9780060985080,en,2003-10-01 00:00:00,4.10,1951.0,Rosalind Franklin : The Dark Lady of DNA,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108948,[653819],3112983.0,"[3092, 2521, 2543, 2564]",The man who conceived of the Appalachian Trail believed that time ...,1.0,9781482446739,en,2016-08-15 00:00:00,NaN,NaN,The Appalachian Trail,1,1
1109075,[624266],NaN,"[3013, 3055]","Offers a compelling account of the dramatic story of Ruffian, hail...",1.0,9780345386021,en,1994-04-01 00:00:00,4.36,2571.0,Ruffian: Burning from the Start,1,1
1109118,[232677],NaN,[3055],NaN,1.0,9780026887588,NaN,1990-05-01 00:00:00,4.20,24654.0,King of the Wind,1,1
1109341,[35294],NaN,[3096],This planner has all you need to organize your life in 2019! Featu...,1.0,9781723791123,en,2018-09-18 00:00:00,NaN,NaN,I,1,1


In [236]:
df_Crossing_book.head(10)

,Unnamed: 0,user_id,age,rating,title,author,year_of_publication,publisher,Summary,Language,Category,city,state,country,depository_book_exists
0,0,2,18.0000,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,Provides an introduction to classical myths placing the addressed\...,en,['Social Science'],stockton,california,usa,1
15,15,8,34.7439,0,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],timmins,ontario,canada,1
16,16,2954,71.0000,8,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],wichita,kansas,usa,1
17,17,152827,40.0000,7,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,"Here, for the first time in paperback, is an outstanding military\...",en,['1940-1949'],ottawa,ontario,canada,1
18,18,8,34.7439,0,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],timmins,ontario,canada,1
19,19,35704,53.0000,6,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],kansas city,missouri,usa,1
20,20,83160,65.0000,0,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],oregon city,oregon,usa,1
21,21,110912,36.0000,10,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],milpitas,california,usa,1
22,22,137688,46.0000,0,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],medford,new york,usa,1
23,23,157969,30.0000,8,Flu : The Story of the Great Influenza Pandemic of 1918 and the Se...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an outbreak that killed ...",en,['Medical'],denver,colorado,usa,1
